In [15]:
import pandas as pd
import numpy as np

In [16]:
data_student=pd.read_csv("latest_data.csv")

In [17]:
data_student=data_student.head(4500)
data_student.head(5)

,Unnamed: 0,Question,Answer,Sentiment (Positive / Negative / Neutral / Ambiguous),Unnamed: 4
0,461129,5) What do you think were the strongest featur...,Having the option to be able to do our work wh...,Positive,NaN
1,62575,Please provide an example and/or explanation t...,The instructor's lectures were clear and organ...,Neutral,NaN
2,550133,What did the instructor do well?\n,The instructor explained the content of the co...,Positive,NaN
3,185121,Your instructor would like to know whether the...,Overall she did really well. I understood all ...,Neutral,NaN
4,384420,Comments:\n,good\n,Positive,NaN


In [18]:

data_student.describe()

,Unnamed: 0
count,4500.000000
mean,301735.622444
std,172354.751507
min,71.000000
25%,158139.000000
50%,302335.500000
75%,450486.000000
max,602946.000000


In [19]:
data_student=data_student.drop("Unnamed: 0", axis=1)
data_student=data_student.drop("Unnamed: 4", axis=1)
data_student=data_student.drop("Question", axis=1)

In [20]:

data_student = data_student.rename(columns = {'Sentiment (Positive / Negative / Neutral / Ambiguous)': 'Sentiment'}, inplace = False)

In [21]:
data_student.describe()

,Answer,Sentiment
count,4500,4500
unique,4336,4
top,<Faculty Name>,Positive
freq,24,3018


In [22]:
data_student.groupby("Sentiment").describe()

Answer                            
           count unique             top freq
Sentiment                                   
Ambiguous    152    104  <Faculty Name>   23
Negative     565    562            na\n    3
Neutral      765    753       nothing\n    3
Positive    3018   2943            no\n   10

In [23]:
import string
import nltk

In [24]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kunal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kunal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [25]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [26]:
stopwords.words('english')[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [27]:
lemmatizer = WordNetLemmatizer()
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]
    

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    lemmatized_words = [lemmatizer.lemmatize(word, pos = "v") for word in nopunc.split()]
    # Now just remove any stopwords
    return [word for word in lemmatized_words  if word.lower() not in (stopwords.words('english') or ['teacher','instructor']]

In [28]:
data_student['Answer'].head(5).apply(text_process)

0    [option, able, work, time, able, discussion, b...
1    [instructors, lecture, clear, organize, time, ...
2    [instructor, explain, content, course, well, m...
3    [Overall, really, well, understand, material, ...
4                                               [good]
Name: Answer, dtype: object

In [29]:
data_student['Answer'].head(5)

0    Having the option to be able to do our work wh...
1    The instructor's lectures were clear and organ...
2    The instructor explained the content of the co...
3    Overall she did really well. I understood all ...
4                                               good\n
Name: Answer, dtype: object

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

In [239]:
bow_transformer = CountVectorizer(analyzer=txt_process).fit(data_student['Answer'])

# Print total number of vocab words
print(len(bow_transformer.vocabulary_))

4077


In [240]:
message = data_student['Answer'][17]
print(message)

Maybe don't have lectures be really long.



In [241]:
bow17 = bow_transformer.transform([message])
print(bow17)
print(bow17.shape)#checking which words are used in 11th Answer

  (0, 1097)	1
  (0, 1690)	1
  (0, 2106)	1
  (0, 2178)	1
  (0, 2247)	1
  (0, 2965)	1
(1, 4077)


In [242]:
print(bow_transformer.get_feature_names()[2000])
print(bow_transformer.get_feature_names()[3000])#word of 2000 and 3000 vocab

iterate
refuse


In [243]:
Answers_bow = bow_transformer.transform(data_student['Answer'])#Sparse Matrix is created

In [244]:
print('Shape of Sparse Matrix: ', Answers_bow.shape)
print('Amount of Non-Zero occurences: ', Answers_bow.nnz)

Shape of Sparse Matrix:  (4500, 4077)
Amount of Non-Zero occurences:  76922


In [245]:
sparsity = (100.0 * Answers_bow.nnz / (Answers_bow.shape[0] * Answers_bow.shape[1]))
print('sparsity: {}'.format(round(sparsity)))

sparsity: 0


In [246]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(Answers_bow)
tfidf17 = tfidf_transformer.transform(bow17)
print(tfidf17)

  (0, 2965)	0.3408343770690091
  (0, 2247)	0.4845622852054888
  (0, 2178)	0.5113179309348138
  (0, 2106)	0.30932791562150086
  (0, 1690)	0.2480053657540116
  (0, 1097)	0.47999465814051795


In [247]:
Answers_tfidf = tfidf_transformer.transform(Answers_bow)
print(Answers_tfidf.shape)

(4500, 4077)


In [248]:
from sklearn.model_selection import train_test_split

In [479]:
ans_train, ans_test, Sentiment_train, Sentiment_test = \
train_test_split(data_student['Answer'], data_student['Sentiment'], test_size=0.1)


In [42]:
from sklearn.svm import SVC

In [43]:
model = SVC()

In [480]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=txt_process)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', SVC()),  # train on TF-IDF vectors w/ SVM
])

In [481]:
pipeline.fit(ans_train,Sentiment_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function txt_process at 0x000001350FD5B0D0>)),
                ('tfidf', TfidfTransformer()), ('classifier', SVC())])

In [497]:
predictionsvc = pipeline.predict(ans_test)

In [47]:
from sklearn.metrics import classification_report

In [483]:
print(classification_report(predictionsvc,Sentiment_test))

              precision    recall  f1-score   support

   Ambiguous       0.29      0.71      0.42         7
    Negative       0.16      0.58      0.25        12
     Neutral       0.10      0.50      0.16        14
    Positive       0.97      0.74      0.84       417

    accuracy                           0.73       450
   macro avg       0.38      0.63      0.42       450
weighted avg       0.91      0.73      0.80       450



In [49]:
Sentiment_test.describe()

count         1350
unique           4
top       Positive
freq           894
Name: Sentiment, dtype: object

In [50]:
from sklearn.tree import DecisionTreeClassifier

In [51]:
dtree = DecisionTreeClassifier()

In [527]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('model', DecisionTreeClassifier()),  # train on TF-IDF vectors w/ Decision Tree
])

In [528]:
pipeline.fit(ans_train,Sentiment_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x00000135739B8940>)),
                ('tfidf', TfidfTransformer()),
                ('model', DecisionTreeClassifier())])

In [535]:
predictions = pipeline.predict(ans_test)

In [530]:
print(classification_report(predictions,Sentiment_test))

              precision    recall  f1-score   support

   Ambiguous       0.35      0.21      0.26        29
    Negative       0.21      0.16      0.18        57
     Neutral       0.22      0.26      0.24        62
    Positive       0.73      0.76      0.74       302

    accuracy                           0.58       450
   macro avg       0.38      0.35      0.36       450
weighted avg       0.57      0.58      0.57       450



In [56]:
from sklearn.ensemble import RandomForestClassifier

In [57]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', RandomForestClassifier()),  # train on TF-IDF vectors w/ Random Forest
])

In [58]:
pipeline.fit(ans_train,Sentiment_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x00000135739B8940>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', RandomForestClassifier())])

In [59]:
predictions = pipeline.predict(ans_test)

In [60]:
print(classification_report(predictions,Sentiment_test))

              precision    recall  f1-score   support

   Ambiguous       0.57      0.42      0.49        66
    Negative       0.08      0.48      0.14        29
     Neutral       0.08      0.39      0.14        51
    Positive       0.96      0.71      0.82      1204

    accuracy                           0.68      1350
   macro avg       0.42      0.50      0.40      1350
weighted avg       0.89      0.68      0.76      1350



In [61]:
from sklearn.neighbors import sClassKNeighborifier

In [405]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', KNeighborsClassifier()),  # train on TF-IDF vectors w/ KNN
])

In [406]:
pipeline.fit(ans_train,Sentiment_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x00000135739B8940>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', KNeighborsClassifier())])

In [64]:
predictions = pipeline.predict(ans_test)

In [65]:
print(classification_report(predictions,Sentiment_test))

              precision    recall  f1-score   support

   Ambiguous       0.80      0.05      0.09       856
    Negative       0.00      0.00      0.00         4
     Neutral       0.08      0.24      0.12        84
    Positive       0.35      0.77      0.48       406

    accuracy                           0.27      1350
   macro avg       0.31      0.26      0.17      1350
weighted avg       0.61      0.27      0.21      1350



In [254]:
from sklearn.neural_network import MLPClassifier

In [255]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MLPClassifier()),  # train on TF-IDF vectors w/ multi-layer perceptron (MLP)
])

In [256]:
pipeline.fit(ans_train,Sentiment_train)

C:\Users\kunal\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x00000135739B8940>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', MLPClassifier())])

In [257]:
predictionsmlp = pipeline.predict(ans_test)

In [258]:
print(classification_report(predictionsmlp,Sentiment_test))

              precision    recall  f1-score   support

   Ambiguous       0.46      0.38      0.41        32
    Negative       0.27      0.34      0.30        92
     Neutral       0.22      0.22      0.22       145
    Positive       0.79      0.77      0.78       631

    accuracy                           0.62       900
   macro avg       0.44      0.43      0.43       900
weighted avg       0.63      0.62      0.63       900



In [407]:
param_grid = {'C': [0.1,1, 10, 100, 1000], 'gamma': [1,0.1,0.01,0.001,0.0001], 'kernel': ['rbf']} 

In [408]:
from sklearn.model_selection import GridSearchCV

In [409]:
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=3)

In [410]:
# May take awhile!
grid.fit(ans_train,Sentiment_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ............ C=0.1, gamma=1, kernel=rbf, score=nan, total=   0.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ............ C=0.1, gamma=1, kernel=rbf, score=nan, total=   0.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ............ C=0.1, gamma=1, kernel=rbf, score=nan, total=   0.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ............ C=0.1, gamma=1, kernel=rbf, score=nan, total=   0.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ............ C=0.1, gamma=1, kernel=rbf, score=nan, total=   0.0s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] .......... C=0.1, gamma=0.1, kernel=rbf, score=nan, total=   0.0s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ..........

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\kunal\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\kunal\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\kunal\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 160, in fit
    X, y = self._validate_data(X, y, dtype=np.float64,
  File "C:\Users\kunal\anaconda3\lib\site-packages\sklearn\base.py", line 432, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "C:\Users\kunal\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 72, in inner_f
    return f(**kwargs)
  File "C:\Users\kunal\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 7

C:\Users\kunal\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\kunal\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\kunal\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 160, in fit
    X, y = self._validate_data(X, y, dtype=np.float64,
  File "C:\Users\kunal\anaconda3\lib\site-packages\sklearn\base.py", line 432, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "C:\Users\kunal\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 72, in inner_f
    return f(**kwargs)
  File "C:\Users\kunal\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 795, in check_X_y
    X = check_array(X, accept_sparse=accept_sparse,
  File "C:\U

[CV] ....... C=0.1, gamma=0.0001, kernel=rbf, score=nan, total=   0.0s
[CV] C=0.1, gamma=0.0001, kernel=rbf .................................
[CV] ....... C=0.1, gamma=0.0001, kernel=rbf, score=nan, total=   0.0s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] .............. C=1, gamma=1, kernel=rbf, score=nan, total=   0.0s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] .............. C=1, gamma=1, kernel=rbf, score=nan, total=   0.0s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] .............. C=1, gamma=1, kernel=rbf, score=nan, total=   0.0s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] .............. C=1, gamma=1, kernel=rbf, score=nan, total=   0.0s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] .............. C=1, gamma=1, kernel=rbf, score=nan, total=   0.0s
[CV] C=1, gamma=0.1, kernel=rbf ......................................
[CV] .

[CV] ...... C=1000, gamma=0.0001, kernel=rbf, score=nan, total=   0.0s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] ...... C=1000, gamma=0.0001, kernel=rbf, score=nan, total=   0.0s


C:\Users\kunal\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\kunal\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\kunal\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 160, in fit
    X, y = self._validate_data(X, y, dtype=np.float64,
  File "C:\Users\kunal\anaconda3\lib\site-packages\sklearn\base.py", line 432, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "C:\Users\kunal\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 72, in inner_f
    return f(**kwargs)
  File "C:\Users\kunal\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 795, in check_X_y
    X = check_array(X, accept_sparse=accept_sparse,
  File "C:\U

ValueError: could not convert string to float: 'yes\n'

In [580]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=txt_process)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', GridSearchCV(SVC(),param_grid,refit=True,verbose=3)),  # train on TF-IDF vectors w/ multi-layer perceptron (MLP)
])

In [581]:
pipeline.fit(ans_train,Sentiment_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.668, total=   3.2s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.1s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.667, total=   2.4s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


...

[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.667, total=   2.5s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.667, total=   2.4s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.667, total=   2.4s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.668, total=   1.8s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.667, total=   1.8s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.667, total=   1.9s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.667, total=   1.9s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] .

[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.668, total=   1.8s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.667, total=   1.7s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.667, total=   1.9s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.667, total=   1.8s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.667, total=   1.8s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] ....... C=10, gamma=0.001, kernel=rbf, score=0.668, total=   1.8s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] ....... C=10, gamma=0.001, kernel=rbf, score=0.667, total=   1.8s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] .

[CV] ..... C=1000, gamma=0.001, kernel=rbf, score=0.700, total=   2.1s
[CV] C=1000, gamma=0.001, kernel=rbf .................................
[CV] ..... C=1000, gamma=0.001, kernel=rbf, score=0.705, total=   2.2s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] .... C=1000, gamma=0.0001, kernel=rbf, score=0.668, total=   2.2s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] .... C=1000, gamma=0.0001, kernel=rbf, score=0.667, total=   2.3s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] .... C=1000, gamma=0.0001, kernel=rbf, score=0.667, total=   2.2s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] .... C=1000, gamma=0.0001, kernel=rbf, score=0.667, total=   2.1s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] .... C=1000, gamma=0.0001, kernel=rbf, score=0.667, total=   2.0s


[Parallel(n_jobs=1)]: Done 125 out of 125 | elapsed:  4.3min finished


Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function txt_process at 0x0000013514D5A700>)),
                ('tfidf', TfidfTransformer()),
                ('classifier',
                 GridSearchCV(estimator=SVC(),
                              param_grid={'C': [0.1, 1, 10, 100, 1000],
                                          'gamma': [1, 0.1, 0.01, 0.001,
                                                    0.0001],
                                          'kernel': ['rbf']},
                              verbose=3))])

In [582]:
predictions = pipeline.predict(ans_test)

In [583]:
print(classification_report(predictions,Sentiment_test))

              precision    recall  f1-score   support

   Ambiguous       0.29      0.56      0.38         9
    Negative       0.26      0.39      0.31        28
     Neutral       0.23      0.42      0.30        40
    Positive       0.93      0.79      0.85       373

    accuracy                           0.73       450
   macro avg       0.43      0.54      0.46       450
weighted avg       0.81      0.73      0.76       450



In [289]:
s=["worse teacher"]

In [290]:
predictionsmlp = pipeline.predict(s)

In [291]:
predictionsmlp

array(['Positive'], dtype='<U9')

In [213]:

print(bow_transformer.get_feature_names()[99])

800


In [292]:
bnow = bow_transformer.transform(['teacher'])
print(bnow)
print(bnow.shape)#checking which words are used in 11th Answer


(1, 4077)


In [141]:
bnow = bow_transformer.transform(['bad'])
print(bnow)
print(bnow.shape)#checking which words are used in 11th Answer

  (0, 1425)	1
(1, 4737)


In [146]:
bnow = bow_transformer.transform(['never'])
print(bnow)
print(bnow.shape)#checking which words are used in 11th Answer

  (0, 3242)	1
(1, 4737)


TypeError: 'NoneType' object is not callable

In [238]:
lemmatizer = WordNetLemmatizer()
def txt_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    mess=mess.lower()
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]
    

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    lemmatized_words = [lemmatizer.lemmatize(word, pos = "v") for word in nopunc.split()]
    # Now just remove any stopwords
    return [word for word in lemmatized_words  if word.lower() not in (stopwords.words('english') and ['teacher','instructor','time','the','she','he','do','be','of','to'] )]

In [237]:
data_student['Answer'].head(5).apply(txt_process)

0    [have, option, to, able, to, our, work, when, ...
1    [instructors, lecture, clear, and, organize, b...
2    [explain, content, course, well, and, make, it...
3    [overall, really, well, i, understand, all, if...
4                                               [good]
Name: Answer, dtype: object

{'option': 3345, 'able': 1146, 'work': 4681, 'time': 4363, 'discussion': 2010, 'board': 1490, 'students': 4178, 'order': 3349, 'see': 3950, 'think': 4336, 'subject': 4187, 'matter': 3085, 'instructors': 2807, 'lecture': 2959, 'clear': 1661, 'organize': 3357, 'would': 4702, 'skip': 4041, 'many': 3066, 'slide': 4044, 'expect': 2274, 'go': 2521, 'Also': 146, 'make': 3051, 'available': 1405, 'different': 1981, 'ones': 3324, 'present': 3579, 'class': 1649, 'change': 1605, 'information': 2767, 'instructor': 2806, 'explain': 2285, 'content': 1802, 'course': 1845, 'well': 4629, 'easier': 2093, 'understand': 4476, 'Overall': 761, 'really': 3725, 'material': 3078, 'like': 2995, 'give': 2512, 'study': 4181, 'guide': 2560, 'plenty': 3505, 'days': 1898, 'review': 3856, 'Throughout': 1035, 'teach': 4272, 'could': 1837, 'tell': 4287, 'prepare': 3574, 'good': 2528, 'demand': 1938, 'read': 3715, 'regardless': 3760, 'COVID19': 241, 'Dr': 345, 'Faculty': 427, 'Name': 712, 'poor': 3521, 'response': 3836, 

In [221]:
Answers_bow.

AttributeError: all not found

In [231]:
messa="I Love me"
messa=messa.lower()
messa        

'i love me'

In [293]:
type(['teacher'])

list

In [294]:
my_file = open("Bad.txt", "r")
content_list = my_file. readlines()

In [313]:
for word in content_list:
    if word=='abort\n':
        print('Hi')

Hi


In [300]:
t='abnormal\n'

In [303]:
t=t.strip('\n')

In [320]:
bad_temp = open('Bad.txt','r').read().split('\n')

In [321]:
good_temp = open('Good.txt','r').read().split('\n')

In [362]:
lemmatizer = WordNetLemmatizer()
def txt_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    mess=mess.lower()
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]
    

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    lemmatized_words = [lemmatizer.lemmatize(word, pos = "v") for word in nopunc.split()]
    # Now just remove any stopwords
    return [word for word in lemmatized_words  if word.lower() not in (stopwords.words('english') and ['teacher','instructor','time','the','she','he','do','be')]

In [363]:
datan=data_student['Answer'].apply(txt_process)

In [364]:
datan.head()

0    [have, option, to, able, to, our, work, when, ...
1    [instructors, lecture, clear, and, organize, b...
2    [explain, content, of, course, well, and, make...
3    [overall, really, well, i, understand, all, if...
4                                               [good]
Name: Answer, dtype: object

In [373]:
def calcu(messlist):
    good=0
    bad=0
    for word in messlist:
        if word in bad_temp:
            bad=bad+1
        if word in good_temp:
            good=good+1
        if word == "dont":
            bad=bad+2
    if good>bad:
        return "Positive"
    if good<bad:
        return "Negative"
    if good==bad and good>0 :
        return "Neutral"
    return "Ambiguous"

In [378]:
datan.head(5).apply(calcu)

0    Positive
1    Positive
2    Positive
3    Positive
4    Positive
Name: Answer, dtype: object

In [375]:
n="I don't like this teacher"

In [376]:
calcu(txt_process(n))

'Negative'

In [377]:
txt_process(n)

['i', 'dont', 'like', 'this']

In [379]:
predictionsmlp = pipeline.predict(ans_test)

In [380]:
type(predictionsmlp)

numpy.ndarray

In [663]:
other=ans_test.apply(txt_process)

In [664]:
otherpred=other.apply(calcu)

In [590]:
mixe=predictions
for i in range(0,ans_test.size):
    if predictions[i]=='Positive' and (nn[i]=='Positive'):
        mixe[i]=nn[i]
    if predictions[i]=='Neutral' and nn[i]=='Ambiguous':
        continue
    if predictions[i]!='Negative' and nn[i]=='Negative':
        mixe[i]=nn[i]
        
    

In [665]:
nn=otherpred.array

In [591]:
predictions

array(['Negative', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Negative', 'Positive', 'Negative',
       'Positive', 'Neutral', 'Positive', 'Neutral', 'Negative',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Neutral',
       'Negative', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Negative', 'Ambiguous',
       'Negative', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Negative', 'Positive', 'Positive',
       'Positive', 'Positive', 'Negative', 'Negative', 'Positive',
       'Neutral', 'Positive', 'Positive', 'Positive', 'Positive',
       'Negative', 'Positive', 'Neutral', 'Positive', 'Positive',
       'Positive', 'Negative', 'Positive', 'Positive', 'Positive',
       'Positive', 'Neutral', 'Positive', 'Neutral', 'Positive',
 

In [431]:
print(classification_report(predictionsmlp,Sentiment_test))

              precision    recall  f1-score   support

   Ambiguous       0.62      0.10      0.18       155
    Negative       0.32      0.27      0.29       137
     Neutral       0.26      0.22      0.24       175
    Positive       0.59      0.84      0.69       433

    accuracy                           0.50       900
   macro avg       0.45      0.36      0.35       900
weighted avg       0.49      0.50      0.45       900



In [402]:
s="He is a good teacher"

In [594]:
Sentiment_test.describe()

count          450
unique           4
top       Positive
freq           317
Name: Sentiment, dtype: object

In [593]:
print(classification_report(mixe,Sentiment_test))

              precision    recall  f1-score   support

   Ambiguous       0.29      0.56      0.38         9
    Negative       0.56      0.33      0.41        73
     Neutral       0.16      0.40      0.23        30
    Positive       0.88      0.82      0.85       338

    accuracy                           0.71       450
   macro avg       0.47      0.53      0.47       450
weighted avg       0.77      0.71      0.73       450



In [501]:
print(classification_report(predictionsvc,Sentiment_test))

              precision    recall  f1-score   support

   Ambiguous       0.47      0.09      0.15        92
    Negative       0.44      0.33      0.38        58
     Neutral       0.19      0.37      0.25        38
    Positive       0.70      0.85      0.77       262

    accuracy                           0.58       450
   macro avg       0.45      0.41      0.39       450
weighted avg       0.58      0.58      0.55       450



In [476]:
count=0
for word in nn:
    if word=="Ambiguous":
        count=count+1
count        

219

In [538]:
count=0
for word in Sentiment_test:
    if word=="Neutral":
        count=count+1
count        

73

In [666]:
print(classification_report(nn,Sentiment_test))

              precision    recall  f1-score   support

   Ambiguous       0.59      0.07      0.13       139
    Negative       0.35      0.33      0.34        45
     Neutral       0.10      0.35      0.15        20
    Positive       0.67      0.86      0.75       246

    accuracy                           0.54       450
   macro avg       0.42      0.40      0.34       450
weighted avg       0.58      0.54      0.49       450



In [495]:
nn

<PandasArray>
[ 'Positive',  'Positive',  'Positive',  'Positive',  'Positive',  'Positive',
 'Ambiguous', 'Ambiguous',  'Positive',  'Positive',
 ...
 'Ambiguous',  'Positive',  'Positive',  'Positive',  'Positive',   'Neutral',
  'Negative',  'Negative',  'Positive',  'Positive']
Length: 450, dtype: object

In [548]:
i=0
for index in newans.index:
    
    if Sentiment_test[index] == "Negative":
        print(ans_test[index]+"\n")
    i=i+1    

The professor was very unclear, unorganized, and did not explain the subject well enough for students to learn. She did not give any
feedback or post any grades so I have no idea of how I'm doing in the class. She assigned a lot of reading weekly and the workload was
really heavy for this class. This professor was one of the most difficult ones I've had and wish I would have been able to take this course
with another professor.


listen to students, pick more relevant problems for exams, be more attentive to students struggles


No, I do not think that she was a very goos Professor.


I wish he could teach in a slower phase and use less power point and more interactive. 


na


It was very hard to follow his lectures. Professor had a hard time connecting to zoom every week. Test were unorganized and had wrong
answers.


Yes it was because if you mess up in an exam it can be difficult to pass the class like me for example. However after trying harder in. The
course I was able to get my 

In [517]:
newans=ans_test

In [521]:
newans.index

Int64Index([1835, 2590, 1549,  946,  517, 2389, 1477, 2579, 4173, 1779,
            ...
            2402, 2801, 2680, 3546, 3479, 3187,  622, 2372,  896, 2528],
           dtype='int64', length=450)

In [565]:
final_stop=stopwords.words('english')

In [539]:
predictions = pipeline.predict(ans_test)

In [597]:
i=0
for index in newans.index:
    
    if mixe[i] == "Negative":
        print(ans_test[index]+"\n")
    i=i+1    

Some modules required us to be in class by 8 sharp and would not collect the homework, thus give attendance, if we were even 1 minute
late. This discouraged me from coming to class as there were a couple days where I would be 1 to 2 minutes late. Otherwise the study
guides and packets were very helpful for the end of the module test.


The professor was very unclear, unorganized, and did not explain the subject well enough for students to learn. She did not give any
feedback or post any grades so I have no idea of how I'm doing in the class. She assigned a lot of reading weekly and the workload was
really heavy for this class. This professor was one of the most difficult ones I've had and wish I would have been able to take this course
with another professor.


Do not appreciate the 10 page essay in lieu of final exam during this difficult time.


listen to students, pick more relevant problems for exams, be more attentive to students struggles


Personally i feel that he should've bee

In [579]:
lemmatizer = WordNetLemmatizer()
def txt_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    mess=mess.lower()
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]
    

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    lemmatized_words = [lemmatizer.lemmatize(word, pos = "v") for word in nopunc.split()]
    # Now just remove any stopwords
    return [word for word in lemmatized_words  if word.lower() not in (final_stop and ['teacher','instructor','time','the','she','he','do','be'])]

In [574]:
final_stop.remove('not')

In [576]:
final_stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [598]:
data_student

,Answer,Sentiment
0,Having the option to be able to do our work wh...,Positive
1,The instructor's lectures were clear and organ...,Neutral
2,The instructor explained the content of the co...,Positive
3,Overall she did really well. I understood all ...,Neutral
4,good\n,Positive
...,...,...
4495,Professor <Faculty Name> is an excellent profe...,Positive
4496,It wasn't as challenging but definitely a lot ...,Positive
4497,I believe she does well in her grading process...,Positive
4498,"As an instructor, Professor Mills did an amazi...",Positive


In [619]:
lemmatizer = WordNetLemmatizer()
def txt_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    mess=mess.lower()
    # Check characters to see if they are in punctuation
    #nopunc = [char for char in mess if char not in string.punctuation]
    

    # Join the characters again to form the string.
    
    lemmatized_words = [lemmatizer.lemmatize(word, pos = "v") for word in mess.split()]
    # Now just remove any stopwords
    return [word for word in lemmatized_words  if word.lower() not in (final_stop and ['teacher','instructor','time','the','she','he','be'])]

In [600]:
bad_temp.

['abnormal',
 'abolish',
 'abominable',
 'abominably',
 'abominate',
 'abomination',
 'abort',
 'aborted',
 'aborts',
 'abrade',
 'abrasive',
 'abrupt',
 'abruptly',
 'abscond',
 'absence',
 'absent-minded',
 'absentee',
 'absurd',
 'absurdity',
 'absurdly',
 'absurdness',
 'abuse',
 'abused',
 'abuses',
 'abusive',
 'abysmal',
 'abysmally',
 'abyss',
 'accidental',
 'accost',
 'accursed',
 'accusation',
 'accusations',
 'accuse',
 'accuses',
 'accusing',
 'accusingly',
 'acerbate',
 'acerbic',
 'acerbically',
 'ache',
 'ached',
 'aches',
 'achey',
 'aching',
 'acrid',
 'acridly',
 'acridness',
 'acrimonious',
 'acrimoniously',
 'acrimony',
 'adamant',
 'adamantly',
 'addict',
 'addicted',
 'addicting',
 'addicts',
 'admonish',
 'admonisher',
 'admonishingly',
 'admonishment',
 'admonition',
 'adulterate',
 'adulterated',
 'adulteration',
 'adulterier',
 'adversarial',
 'adversary',
 'adverse',
 'adversity',
 'afflict',
 'affliction',
 'afflictive',
 'affront',
 'afraid',
 'aggravate',

In [601]:
final_stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [602]:
some_temp=["don't",
 "aren't",
 'couldn',
 "couldn't",
 'didn',
 "didn't",
 'doesn',
 "doesn't",
 'hadn',
 "hadn't",
 'hasn',
 "hasn't",
 'haven',
 "haven't",
 'isn',
 "isn't",
 'ma',
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't"]

In [604]:
for word in some_temp:
    final_stop.remove(word)

In [605]:
final_stop


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [656]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=txt_process)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MLPClassifier()),  # train on TF-IDF vectors w/ multi-layer perceptron (MLP)
])

In [674]:
pipeline.fit(ans_train,Sentiment_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function txt_process at 0x0000013514D5AEE0>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', MLPClassifier())])

In [675]:
predictionsmlp = pipeline.predict(ans_test)

In [676]:
print(classification_report(predictionsmlp,Sentiment_test))#mlp

              precision    recall  f1-score   support

   Ambiguous       0.29      0.38      0.33        13
    Negative       0.28      0.28      0.28        43
     Neutral       0.30      0.31      0.31        71
    Positive       0.82      0.80      0.81       323

    accuracy                           0.66       450
   macro avg       0.42      0.44      0.43       450
weighted avg       0.67      0.66      0.67       450



In [651]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=txt_process)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('model', DecisionTreeClassifier()),  # train on TF-IDF vectors w/ Decision Tree
])

In [652]:
pipeline.fit(ans_train,Sentiment_train)
predictions = pipeline.predict(ans_test)

In [653]:
print(classification_report(predictions,Sentiment_test))#dtree

              precision    recall  f1-score   support

   Ambiguous       0.29      0.50      0.37        10
    Negative       0.30      0.22      0.26        58
     Neutral       0.26      0.27      0.26        71
    Positive       0.79      0.80      0.79       311

    accuracy                           0.64       450
   macro avg       0.41      0.45      0.42       450
weighted avg       0.63      0.64      0.63       450



In [654]:
mixe=predictions
for i in range(0,ans_test.size):
    if predictions[i]=='Positive' and (nn[i]=='Positive'):
        mixe[i]=nn[i]
    if predictions[i]=='Neutral' and nn[i]=='Ambiguous':
        continue
    if predictions[i]!='Negative' and nn[i]=='Negative':
        mixe[i]=nn[i]
        
    

In [655]:
print(classification_report(mixe,Sentiment_test))#dtree

              precision    recall  f1-score   support

   Ambiguous       0.29      0.50      0.37        10
    Negative       0.56      0.25      0.35        96
     Neutral       0.19      0.24      0.21        58
    Positive       0.74      0.83      0.78       286

    accuracy                           0.62       450
   macro avg       0.45      0.45      0.43       450
weighted avg       0.62      0.62      0.61       450



In [677]:
#predictionsmlp
for i in range(0,ans_test.size):
    if predictionsmlp[i]=='Positive' and (nn[i]=='Positive'):
        predictionsmlp[i]=nn[i]
    if predictionsmlp[i]=='Neutral' and nn[i]=='Ambiguous':
        continue
    if predictionsmlp[i]!='Negative' and nn[i]=='Negative':
        predictionsmlp[i]=nn[i]
        
    

In [678]:
print(classification_report(predictionsmlp,Sentiment_test))#mlp

              precision    recall  f1-score   support

   Ambiguous       0.29      0.45      0.36        11
    Negative       0.53      0.29      0.38        78
     Neutral       0.23      0.29      0.26        58
    Positive       0.80      0.84      0.82       303

    accuracy                           0.66       450
   macro avg       0.47      0.47      0.45       450
weighted avg       0.67      0.66      0.66       450



In [634]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=txt_process)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', GridSearchCV(SVC(),param_grid,refit=True,verbose=3)),  # train on TF-IDF vectors w/ multi-layer perceptron (MLP)
])

In [635]:
pipeline.fit(ans_train,Sentiment_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.673, total=   2.6s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.5s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.674, total=   2.6s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


...

[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.667, total=   2.7s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.672, total=   2.6s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.670, total=   2.5s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.668, total=   1.9s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.667, total=   1.9s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.667, total=   1.9s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.667, total=   1.9s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] .

[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.679, total=   1.8s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.675, total=   1.8s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.670, total=   1.9s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.672, total=   1.8s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.675, total=   1.8s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] ....... C=10, gamma=0.001, kernel=rbf, score=0.668, total=   1.8s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] ....... C=10, gamma=0.001, kernel=rbf, score=0.667, total=   1.8s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] .

[CV] ..... C=1000, gamma=0.001, kernel=rbf, score=0.681, total=   1.9s
[CV] C=1000, gamma=0.001, kernel=rbf .................................
[CV] ..... C=1000, gamma=0.001, kernel=rbf, score=0.690, total=   1.9s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] .... C=1000, gamma=0.0001, kernel=rbf, score=0.679, total=   1.8s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] .... C=1000, gamma=0.0001, kernel=rbf, score=0.675, total=   1.8s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] .... C=1000, gamma=0.0001, kernel=rbf, score=0.670, total=   1.8s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] .... C=1000, gamma=0.0001, kernel=rbf, score=0.672, total=   1.8s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] .... C=1000, gamma=0.0001, kernel=rbf, score=0.675, total=   1.8s


[Parallel(n_jobs=1)]: Done 125 out of 125 | elapsed:  4.3min finished


Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function txt_process at 0x0000013514D5AEE0>)),
                ('tfidf', TfidfTransformer()),
                ('classifier',
                 GridSearchCV(estimator=SVC(),
                              param_grid={'C': [0.1, 1, 10, 100, 1000],
                                          'gamma': [1, 0.1, 0.01, 0.001,
                                                    0.0001],
                                          'kernel': ['rbf']},
                              verbose=3))])

In [636]:
predictionsgs = pipeline.predict(ans_test)

In [668]:
print(classification_report(predictionsgs,Sentiment_test))#grid seach

              precision    recall  f1-score   support

   Ambiguous       0.29      0.56      0.38         9
    Negative       0.53      0.33      0.41        69
     Neutral       0.15      0.34      0.21        32
    Positive       0.87      0.81      0.84       340

    accuracy                           0.70       450
   macro avg       0.46      0.51      0.46       450
weighted avg       0.76      0.70      0.72       450



In [638]:
mixe=predictionsgs
for i in range(0,ans_test.size):
    if predictionsgs[i]=='Positive' and (nn[i]=='Positive'):
        mixe[i]=nn[i]
    if predictionsgs[i]=='Neutral' and nn[i]=='Ambiguous':
        continue
    if predictionsgs[i]!='Negative' and nn[i]=='Negative':
        mixe[i]=nn[i]
        
    

In [667]:
print(classification_report(mixe,Sentiment_test))#gs

              precision    recall  f1-score   support

   Ambiguous       0.29      0.45      0.36        11
    Negative       0.58      0.29      0.38        87
     Neutral       0.23      0.30      0.26        56
    Positive       0.78      0.83      0.81       296

    accuracy                           0.65       450
   macro avg       0.47      0.47      0.45       450
weighted avg       0.66      0.65      0.65       450



In [662]:
def calcu(messlist):
    good=0
    bad=0
    for word in messlist:
        if word in bad_temp:
            bad=bad+1
        if word in good_temp:
            good=good+1
        if word == "don't":
            bad=bad+2
    if good>bad:
        return "Positive"
    if good<bad:
        return "Negative"
    if good==bad and good>0 :
        return "Neutral"
    return "Ambiguous"

In [688]:
s="i know this guy"
calcu(txt_process(s))

'Ambiguous'